In [1]:
from data_collection.data_collection import LoggerSet, Logger
from multiprocessing import Manager

import numpy as np
import pandas as pd
import plotly.express as px
from data_collection.video_data import get_frame_iterator
import tensorflow as tf
import tensorflow.keras as keras


In [2]:
logger_set = LoggerSet('../log/test6', overwrite_ok=False)
logger_set.export_to_parquet()

100%|██████████| 4/4 [00:00<00:00,  5.08it/s]

some failed


{'AngularSpeedControlV2': EOFError('Ran out of input')}

In [3]:
frames = get_frame_iterator('../log/test5/PicameraV2/video')

In [2]:

def get_model(lr=.1):
    tf.keras.backend.clear_session()
    image_shape = 64, 114, 3
    
    model = keras.Sequential([
        keras.layers.InputLayer(image_shape), 
        keras.layers.Conv2D(16, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),
        
        keras.layers.Conv2D(32, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(64, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),
        
        keras.layers.Conv2D(64, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),
        
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'), 
        keras.layers.Dense(64, activation='relu'), 
        keras.layers.Dense(4, activation='softmax'), 
    ])

    optimiser = keras.optimizers.Adam(lr)
    model.compile(optimizer=optimiser, loss='Huber', metrics=['MAE'])

    return model 

model = get_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 112, 16)    │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 56, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 54, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 27, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 25, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 12, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 10, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,076 (590.14 KB)

 Trainable params: 151,076 (590.14 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
idx, arr = next(frames)

In [7]:
import time
t0 = time.monotonic()
for i in range(100): 
    model(arr[None,:])

t1 = time.monotonic()

100/(t1-t0)

21.802741250022613

In [8]:
next(frames)[1].shape

(64, 114, 3)

In [3]:
model.save('test.keras')
#model = tf.keras.models.load_model('test.keras')
